## Importing Libs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests 
from bs4 import BeautifulSoup
import json
from pprint import pprint
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
import highlight_text
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
from mplsoccer import Pitch, VerticalPitch
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
from matplotlib.patches import Rectangle
import matplotlib.font_manager as font_manager
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
from io import BytesIO
from mplsoccer import PyPizza, add_image, FontManager
#import soccerdata as sd
import re
from unidecode import unidecode

pd.set_option("display.max_rows", None, "display.max_columns", None)
background = "black"
text_color = 'w'
primary = '#C8102E'
secondary = 'lightblue'
mpl.rcParams['xtick.color'] = text_color
mpl.rcParams['ytick.color'] = text_color

In [2]:
import matplotlib.font_manager as fm
from mplsoccer import FontManager

# Set the path to the locally downloaded font file
font_path = r'C:\Users\acer\Documents\GitHub\IndianCitizen\ScorePredict\Score Logos-20241022T100701Z-001\Score Logos\Sora_Font\Sora-Regular.ttf'

# Add the font to matplotlib
font_prop = fm.FontProperties(fname=font_path)

# You can now use this font in your plots
fm_sora = FontManager()
#fm_sora.prop = font_prop

print("Font loaded and registered successfully!")

Font loaded and registered successfully!


## Data Extraction

### WhoScored

In [191]:
teams_dict = {
    13: 'Arsenal',
    161: 'Wolves',
    24: 'Aston Villa',
    211: 'Brighton',
    30: 'Tottenham',
    167: 'Man City',
    14: 'Leicester',
    18: 'Southampton',
    183: 'Bournemouth',
    26: 'Liverpool',
    23: 'Newcastle',
    15: 'Chelsea',
    174: 'Nottingham Forest',
    29: 'West Ham',
    32: 'Man Utd'
}

In [192]:
def extract_json_from_html(html_path, save_output=False):
    with open(html_path, 'r', encoding='utf-8') as html_file:
        html = html_file.read()

    regex_pattern = r'(?<=require\.config\.params\["args"\].=.)[\s\S]*?;'
    data_txt = re.findall(regex_pattern, html)[0]

    # add quotations for JSON parser
    data_txt = data_txt.replace('matchId', '"matchId"')
    data_txt = data_txt.replace('matchCentreData', '"matchCentreData"')
    data_txt = data_txt.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    data_txt = data_txt.replace('formationIdNameMappings', '"formationIdNameMappings"')
    data_txt = data_txt.replace('};', '}')

    if save_output:
        # save JSON data to txt
        output_file = open(f"{html_path}.txt", "wt", encoding='utf-8')
        n = output_file.write(data_txt)
        output_file.close()

    return data_txt

def extract_data_from_dict(data):
    # load data from json
    event_types_json = data["matchCentreEventTypeJson"]
    formation_mappings = data["formationIdNameMappings"]
    events_dict = data["matchCentreData"]["events"]
    teams_dict = {data["matchCentreData"]['home']['teamId']: data["matchCentreData"]['home']['name'],
                  data["matchCentreData"]['away']['teamId']: data["matchCentreData"]['away']['name']}
    players_dict = data["matchCentreData"]["playerIdNameDictionary"]
    # create players dataframe
    players_home_df = pd.DataFrame(data["matchCentreData"]['home']['players'])
    players_home_df["teamId"] = data["matchCentreData"]['home']['teamId']
    players_away_df = pd.DataFrame(data["matchCentreData"]['away']['players'])
    players_away_df["teamId"] = data["matchCentreData"]['away']['teamId']
    players_df = pd.concat([players_home_df, players_away_df])
    players_ids = data["matchCentreData"]["playerIdNameDictionary"]
    return events_dict, players_df, teams_dict

In [193]:
import os

def process_file(i, all_teams_dict):
    match_html_path = f"C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/{i}.html"
    
    try:
        json_data_txt = extract_json_from_html(match_html_path)
        data = json.loads(json_data_txt)
        
        events_dict, players_df, teams_dict = extract_data_from_dict(data)
        
        # Append teams_dict for this file
        all_teams_dict[i] = teams_dict
        
        # Save DataFrames to CSV
        df = pd.DataFrame(events_dict)
        df['teamName'] = df['teamId'].map(teams_dict)
        team_names = list(teams_dict.values())
        opposition_dict = {team_names[i]: team_names[1-i] for i in range(len(team_names))}
        df['oppositionTeamName'] = df['teamName'].map(opposition_dict)
        dfp = pd.DataFrame(players_df)

        os.makedirs(f"C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data", exist_ok=True)
        os.makedirs(f"C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data/Player", exist_ok=True)

        df.to_csv(f"C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data/{i}.csv", index=False)
        dfp.to_csv(f"C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data/Player/{i}.csv", index=False)
        
        print(f"Successfully processed file {match_html_path}")

    except Exception as e:
        print(f"Error processing file {match_html_path}: {e}")

# Main loop to process all files
all_teams_dict = {}  # Dictionary to store teams_dict for all files

for i in range(1, 33):
    process_file(i, all_teams_dict)

Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/1.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/2.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/3.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/4.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/5.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/6.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/7.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/8.html
Successfully processed file C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/9.html
Successfully processed file C:/Users/

In [194]:
import glob
import os

# Specify the directory containing the CSV files
folder_path = "C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data/"

# Use glob to get all the CSV file paths
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# List to hold the DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for file in csv_files:
    dfi = pd.read_csv(file)  # Read the CSV file
    dataframes.append(dfi)   # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dataframes, ignore_index=True)

In [195]:
# Specify the directory containing the CSV files
folder_path = "C:/Users/acer/Documents/GitHub/IndianCitizen/ScorePredict/HTMLs/ManCity/Data/Player/"

# Use glob to get all the CSV file paths
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# List to hold the DataFrames
dataframes = []

# Loop through the CSV files and read them into DataFrames
for file in csv_files:
    dfi = pd.read_csv(file)  # Read the CSV file
    dataframes.append(dfi)   # Append the DataFrame to the list

# Concatenate all DataFrames into a single DataFrame
dfp = pd.concat(dataframes, ignore_index=True)

### Statsbomb

In [95]:
from mplsoccer import Sbopen
parser = Sbopen()
df_match = parser.match(competition_id=9, season_id=281)
#df_match.info()

In [96]:
# iterate through all matches to get the events data
df_matches = {}
df_event = pd.DataFrame()
for i, id in enumerate(df_match['match_id']):
  df_matches[id] = {}
  df_matches[id]['event'], df_matches[id]['related'], df_matches[id]['freeze'], df_matches[id]['tactic'] = parser.event(id)
  df_event = pd.concat([df_event,df_matches[id]['event']])

In [100]:
df_statsbomb = df_event.copy()

## Data Transformation

### WhoScored

In [196]:

# Extract the 'displayName' value
df['type'] = df['type'].str.extract(r"'displayName': '([^']+)")
df['outcomeType'] = df['outcomeType'].str.extract(r"'displayName': '([^']+)")
df['period'] = df['period'].str.extract(r"'displayName': '([^']+)")

# temprary use of typeId of period column
df['period'] = df['period'].replace({'FirstHalf': 1, 'SecondHalf': 2, 'FirstPeriodOfExtraTime': 3, 'SecondPeriodOfExtraTime': 4,
                                     'PenaltyShootout': 5, 'PostGame': 14, 'PreMatch': 16})

# new column for cumulative minutes, This part is taken from the "jakeyk11.github.io" github repository and modified for my use
def cumulative_match_mins(events_df):
    events_out = pd.DataFrame()
    # Add cumulative time to events data, resetting for each unique match
    match_events = events_df.copy()
    match_events['cumulative_mins'] = match_events['minute'] + (1/60) * match_events['second']
    # Add time increment to cumulative minutes based on period of game.
    for period in np.arange(1, match_events['period'].max() + 1, 1):
        if period > 1:
            t_delta = match_events[match_events['period'] == period - 1]['cumulative_mins'].max() - \
                                   match_events[match_events['period'] == period]['cumulative_mins'].min()
        elif period == 1 or period == 5:
            t_delta = 0
        else:
            t_delta = 0
        match_events.loc[match_events['period'] == period, 'cumulative_mins'] += t_delta
    # Rebuild events dataframe
    events_out = pd.concat([events_out, match_events])
    return events_out

df = cumulative_match_mins(df)

# Extracting the carry data and merge it with the main df, This part is also taken from the "jakeyk11.github.io" github repository and modified for my use
def insert_ball_carries(events_df, min_carry_length=3, max_carry_length=60, min_carry_duration=1, max_carry_duration=10):
    events_out = pd.DataFrame()
    # Carry conditions (convert from metres to opta)
    min_carry_length = 3.0
    max_carry_length = 60.0
    min_carry_duration = 1.0
    max_carry_duration = 10.0
    # match_events = events_df[events_df['match_id'] == match_id].reset_index()
    match_events = events_df.reset_index()
    match_carries = pd.DataFrame()

    for idx, match_event in match_events.iterrows():

        if idx < len(match_events) - 1:
            prev_evt_team = match_event['teamId']
            next_evt_idx = idx + 1
            init_next_evt = match_events.loc[next_evt_idx]
            take_ons = 0
            incorrect_next_evt = True

            while incorrect_next_evt:

                next_evt = match_events.loc[next_evt_idx]

                if next_evt['type'] == 'TakeOn' and next_evt['outcomeType'] == 'Successful':
                    take_ons += 1
                    incorrect_next_evt = True

                elif ((next_evt['type'] == 'TakeOn' and next_evt['outcomeType'] == 'Unsuccessful')
                      or (next_evt['teamId'] != prev_evt_team and next_evt['type'] == 'Challenge' and next_evt['outcomeType'] == 'Unsuccessful')
                      or (next_evt['type'] == 'Foul')):
                    incorrect_next_evt = True

                else:
                    incorrect_next_evt = False

                next_evt_idx += 1

            # Apply some conditioning to determine whether carry criteria is satisfied
            same_team = prev_evt_team == next_evt['teamId']
            not_ball_touch = match_event['type'] != 'BallTouch'
            dx = 105*(match_event['endX'] - next_evt['x'])/100
            dy = 68*(match_event['endY'] - next_evt['y'])/100
            far_enough = dx ** 2 + dy ** 2 >= min_carry_length ** 2
            not_too_far = dx ** 2 + dy ** 2 <= max_carry_length ** 2
            dt = 60 * (next_evt['cumulative_mins'] - match_event['cumulative_mins'])
            min_time = dt >= min_carry_duration
            same_phase = dt < max_carry_duration
            same_period = match_event['period'] == next_evt['period']

            valid_carry = same_team & not_ball_touch & far_enough & not_too_far & min_time & same_phase &same_period

            if valid_carry:
                carry = pd.DataFrame()
                prev = match_event
                nex = next_evt

                carry.loc[0, 'eventId'] = prev['eventId'] + 0.5
                carry['minute'] = np.floor(((init_next_evt['minute'] * 60 + init_next_evt['second']) + (
                        prev['minute'] * 60 + prev['second'])) / (2 * 60))
                carry['second'] = (((init_next_evt['minute'] * 60 + init_next_evt['second']) +
                                    (prev['minute'] * 60 + prev['second'])) / 2) - (carry['minute'] * 60)
                carry['teamId'] = nex['teamId']
                carry['x'] = prev['endX']
                carry['y'] = prev['endY']
                carry['expandedMinute'] = np.floor(((init_next_evt['expandedMinute'] * 60 + init_next_evt['second']) +
                                                    (prev['expandedMinute'] * 60 + prev['second'])) / (2 * 60))
                carry['period'] = nex['period']
                carry['type'] = carry.apply(lambda x: {'value': 99, 'displayName': 'Carry'}, axis=1)
                carry['outcomeType'] = 'Successful'
                carry['qualifiers'] = carry.apply(lambda x: {'type': {'value': 999, 'displayName': 'takeOns'}, 'value': str(take_ons)}, axis=1)
                carry['satisfiedEventsTypes'] = carry.apply(lambda x: [], axis=1)
                carry['isTouch'] = True
                carry['playerId'] = nex['playerId']
                carry['endX'] = nex['x']
                carry['endY'] = nex['y']
                carry['blockedX'] = np.nan
                carry['blockedY'] = np.nan
                carry['goalMouthZ'] = np.nan
                carry['goalMouthY'] = np.nan
                carry['isShot'] = np.nan
                carry['relatedEventId'] = nex['eventId']
                carry['relatedPlayerId'] = np.nan
                carry['isGoal'] = np.nan
                carry['cardType'] = np.nan
                carry['isOwnGoal'] = np.nan
                carry['type'] = 'Carry'
                carry['cumulative_mins'] = (prev['cumulative_mins'] + init_next_evt['cumulative_mins']) / 2

                match_carries = pd.concat([match_carries, carry], ignore_index=True, sort=False)

    match_events_and_carries = pd.concat([match_carries, match_events], ignore_index=True, sort=False)
    match_events_and_carries = match_events_and_carries.sort_values(['period', 'cumulative_mins']).reset_index(drop=True)

    # Rebuild events dataframe
    events_out = pd.concat([events_out, match_events_and_carries])

    return events_out

df = insert_ball_carries(df, min_carry_length=3, max_carry_length=60, min_carry_duration=1, max_carry_duration=10)

df = df.reset_index(drop=True)
df['index'] = range(1, len(df) + 1)
df = df[['index'] + [col for col in df.columns if col != 'index']]

# Assign xT values
df_base  = df
dfxT = df_base.copy()
dfxT['qualifiers'] = dfxT['qualifiers'].astype(str)
dfxT = dfxT[(~dfxT['qualifiers'].str.contains('Corner'))]
dfxT = dfxT[(dfxT['type'].isin(['Pass', 'Carry'])) & (dfxT['outcomeType']=='Successful')]


xT = pd.read_csv('https://raw.githubusercontent.com/mckayjohns/youtube-videos/main/data/xT_Grid.csv', header=None) # use this if you don't have your own xT value Grid
# xT = pd.read_csv("/content/xT_Grid.csv", header=None)    # use this if you have your own xT value Grid, then place your file path here
xT = np.array(xT)
xT_rows, xT_cols = xT.shape

dfxT['x1_bin_xT'] = pd.cut(dfxT['x'], bins=xT_cols, labels=False)
dfxT['y1_bin_xT'] = pd.cut(dfxT['y'], bins=xT_rows, labels=False)
dfxT['x2_bin_xT'] = pd.cut(dfxT['endX'], bins=xT_cols, labels=False)
dfxT['y2_bin_xT'] = pd.cut(dfxT['endY'], bins=xT_rows, labels=False)

dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)

dfxT['xT'] = dfxT['end_zone_value_xT'] - dfxT['start_zone_value_xT']
columns_to_drop = ['id', 'eventId', 'minute', 'second', 'teamId', 'x', 'y', 'expandedMinute', 'period', 'outcomeType', 'qualifiers',  'type',
                   'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY', 'relatedEventId', 'relatedPlayerId', 'blockedX', 'blockedY',
                   'goalMouthZ', 'goalMouthY', 'isShot', 'cumulative_mins']
dfxT.drop(columns=columns_to_drop, inplace=True)

df = df.merge(dfxT, on='index', how='left')

# New Column for Team Names and Oppositon TeamNames
#df['teamName'] = df['teamId'].map(teams_dict)
#team_names = list(teams_dict.values())
#opposition_dict = {team_names[i]: team_names[1-i] for i in range(len(team_names))}
#df['oppositionTeamName'] = df['teamName'].map(opposition_dict)

# Reshaping the data from 100x100 to 105x68, as I use the pitch_type='uefa', in the pitch function, you can consider according to your use
df['x'] = df['x']*1.05
df['y'] = df['y']*0.68
df['endX'] = df['endX']*1.05
df['endY'] = df['endY']*0.68
df['goalMouthY'] = df['goalMouthY']*0.68

columns_to_drop = ['height', 'weight', 'age', 'isManOfTheMatch', 'field', 'stats',
                   'subbedInPlayerId', 'subbedOutPeriod',
                   'subbedOutExpandedMinute', 'subbedInPeriod', 'subbedInExpandedMinute', 'subbedOutPlayerId',
                   'teamId']
dfp.drop(columns=columns_to_drop, inplace=True)

# adding player name, shirt no. etc info
dfp = dfp.drop_duplicates(subset=['name'])
df = df.merge(dfp, on='playerId', how='left')

df['qualifiers'] = df['qualifiers'].astype(str)
# Calculating passing distance, to find out progressive pass, this will just show the distance reduced by a pass, then will be able to filter passes which has reduced distance value more than 10yds as a progressive pass
df['prog_pass'] = np.where((df['type'] == 'Pass'),
                           np.sqrt((105 - df['x'])**2 + (34 - df['y'])**2) - np.sqrt((105 - df['endX'])**2 + (34 - df['endY'])**2), 0)
# Calculating carrying distance, to find out progressive carry, this will just show the distance reduced by a carry, then will be able to filter carries which has reduced distance value more than 10yds as a progressive carry
df['prog_carry'] = np.where((df['type'] == 'Carry'),
                            np.sqrt((105 - df['x'])**2 + (34 - df['y'])**2) - np.sqrt((105 - df['endX'])**2 + (34 - df['endY'])**2), 0)
df['pass_or_carry_angle'] = np.degrees(np.arctan2(df['endY'] - df['y'], df['endX'] - df['x']))


# Making all the alphabets in the name as English Alphabets only (for example: Á will be replaced by A)
df['name'] = df['name'].astype(str)
df['name'] = df['name'].apply(unidecode)

# Function to extract short names
def get_short_name(full_name):
    if pd.isna(full_name):
        return full_name
    parts = full_name.split()
    if len(parts) == 1:
        return full_name  # No need for short name if there's only one word
    elif len(parts) == 2:
        return parts[0][0] + ". " + parts[1]
    else:
        return parts[0][0] + ". " + parts[1][0] + ". " + " ".join(parts[2:])

# Applying the function to create 'shortName' column
df['shortName'] = df['name'].apply(get_short_name)

df['qualifiers'] = df['qualifiers'].astype(str)
#columns_to_drop2 = ['id']
#df.drop(columns=columns_to_drop2, inplace=True)

df['period'] = df['period'].replace({1: 'FirstHalf', 2: 'SecondHalf', 3: 'FirstPeriodOfExtraTime', 4: 'SecondPeriodOfExtraTime',
                                     5: 'PenaltyShootout', 14: 'PostGame', 16: 'PreMatch'})


C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3822741489.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['start_zone_value_xT'] = dfxT[['x1_bin_xT', 'y1_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)
C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3822741489.py:157: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dfxT['end_zone_value_xT'] = dfxT[['x2_bin_xT', 'y2_bin_xT']].apply(lambda x: xT[x[1]][x[0]], axis=1)


In [197]:
df = df.sort_values('id').reset_index(drop=True)

In [11]:
df.columns

Index(['index', 'eventId', 'minute', 'second', 'teamId', 'x', 'y',
       'expandedMinute', 'period', 'type', 'outcomeType', 'qualifiers',
       'satisfiedEventsTypes', 'isTouch', 'playerId', 'endX', 'endY',
       'blockedX', 'blockedY', 'goalMouthZ', 'goalMouthY', 'isShot',
       'relatedEventId', 'relatedPlayerId', 'isGoal_x', 'cardType_x',
       'isOwnGoal_x', 'cumulative_mins', 'id', 'teamName_x',
       'oppositionTeamName_x', 'isGoal_y', 'cardType_y', 'isOwnGoal_y',
       'teamName_y', 'oppositionTeamName_y', 'x1_bin_xT', 'y1_bin_xT',
       'x2_bin_xT', 'y2_bin_xT', 'start_zone_value_xT', 'end_zone_value_xT',
       'xT', 'shirtNo', 'name', 'position', 'isFirstEleven', 'prog_pass',
       'prog_carry', 'pass_or_carry_angle', 'shortName'],
      dtype='object')

In [48]:
df['outcomeType'].unique()

array(['Successful', 'Unsuccessful'], dtype=object)

In [198]:
df_passes = df[(df['type'] == 'Pass') & (df['outcomeType'] == 'Successful')]

In [199]:
df_passes['Is_Assist'] = df_passes['qualifiers'].str.contains('IntentionalGoalAssist', na=False)

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\2091178362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['Is_Assist'] = df_passes['qualifiers'].str.contains('IntentionalGoalAssist', na=False)


In [200]:
conditions = [
    df_passes['qualifiers'].str.contains('Throughball', na=False),
    df_passes['qualifiers'].str.contains('Longball', na=False),
    df_passes['qualifiers'].str.contains('Cross', na=False),
    df_passes['qualifiers'].str.contains('HeadPass', na=False)
]

# Define corresponding pass types
pass_types = ['through_pass', 'long_ball', 'cross', 'header']

# Apply np.select to assign Pass_Type based on conditions
df_passes['Pass_Type'] = np.select(conditions, pass_types, default='ground_pass')

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\1813201705.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['Pass_Type'] = np.select(conditions, pass_types, default='ground_pass')


In [201]:
df_passes['IsCorner'] = False

# Find 'Pass' events following 'CornerAwarded' for the same team
for i, row in df_passes.iterrows():
    if (row['type'] == 'CornerAwarded') & (row['outcomeType'] == 'Successful'):
        team = row['teamName_x']
        # Iterate through subsequent rows to find a valid 'Pass' for the same team
        for j in range(i + 1, len(df_passes)):
            if df_passes.loc[j, 'type'] == 'Pass' and df_passes.loc[j, 'teamName_x'] == team:
                df_passes.loc[j, 'IsCorner'] = True
                break

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3051622314.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['IsCorner'] = False


In [202]:
conditions2 = [
    df_passes['qualifiers'].str.contains('ThrowIn', na=False),
    df_passes['qualifiers'].str.contains('Corner', na=False),
    df_passes['qualifiers'].str.contains('Freekick', na=False)
]

# Define corresponding pass types
play_patterns = ['throw_in', 'corner', 'freekick']

# Apply np.select to assign Pass_Type based on conditions
df_passes['Play_Pattern'] = np.select(conditions2, play_patterns, default='open_play')

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\585988565.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['Play_Pattern'] = np.select(conditions2, play_patterns, default='open_play')


In [203]:
df_passes['pass_distance'] = np.sqrt((df_passes['endX'] - df_passes['x'])**2 + 
                                     (df_passes['endY'] - df_passes['y'])**2)

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\97125126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['pass_distance'] = np.sqrt((df_passes['endX'] - df_passes['x'])**2 +


In [204]:
# Compute angle in radians
df_passes['pass_angle'] = np.arctan2(df_passes['endY'] - df_passes['y'], df_passes['endX'] - df_passes['x'])

# Optional: Convert to degrees
#df_passes['pass_angle_degrees'] = np.degrees(df_passes['pass_angle'])


C:\Users\acer\AppData\Local\Temp\ipykernel_23116\2645462579.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['pass_angle'] = np.arctan2(df_passes['endY'] - df_passes['y'], df_passes['endX'] - df_passes['x'])


In [205]:
df_passes['Pass_Type'].fillna('Unknown', inplace=True)
df_passes['Play_Pattern'].fillna('Unknown', inplace=True)

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\1360876484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['Pass_Type'].fillna('Unknown', inplace=True)
C:\Users\acer\AppData\Local\Temp\ipykernel_23116\1360876484.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_passes['Play_Pattern'].fillna('Unknown', inplace=True)


In [206]:
final_df = df_passes[['x', 'y', 'outcomeType', 'endX', 'endY', 'name', 'pass_angle',
                      'Is_Assist', 'Pass_Type', 'Play_Pattern', 'pass_distance']]

In [74]:
final_df['pass_distance'].unique()

array([0.38230286, 0.70664943, 0.20615716, ..., 0.1486492 , 0.12705595,
       0.19926943])

### Statsbomb

In [104]:
df_statsbomb.columns

Index(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'possession',
       'duration', 'match_id', 'type_id', 'type_name', 'possession_team_id',
       'possession_team_name', 'play_pattern_id', 'play_pattern_name',
       'team_id', 'team_name', 'tactics_formation', 'player_id', 'player_name',
       'position_id', 'position_name', 'pass_recipient_id',
       'pass_recipient_name', 'pass_length', 'pass_angle', 'pass_height_id',
       'pass_height_name', 'end_x', 'end_y', 'body_part_id', 'body_part_name',
       'sub_type_id', 'sub_type_name', 'x', 'y', 'under_pressure',
       'outcome_id', 'outcome_name', 'counterpress', 'out', 'aerial_won',
       'off_camera', 'pass_switch', 'pass_cross', 'shot_statsbomb_xg',
       'shot_first_time', 'technique_id', 'technique_name',
       'goalkeeper_position_id', 'goalkeeper_position_name',
       'pass_assisted_shot_id', 'pass_shot_assist', 'end_z',
       'shot_key_pass_id', 'injury_stoppage_in_chain', 'foul_won_defensive',
      

In [116]:
df_statsbomb.shape

(137765, 84)

In [134]:
df_statsbomb['body_part_name'].unique()

array([nan, 'Right Foot', 'Left Foot', 'Other', 'Head', 'Keeper Arm',
       'Drop Kick', 'Both Hands', 'Right Hand', 'Left Hand', 'No Touch',
       'Chest'], dtype=object)

In [182]:
df_passes_sb = df_statsbomb[(df_statsbomb['type_name'] == 'Pass')][['play_pattern_name','pass_angle','pass_length','end_x', 'end_y','sub_type_name','x', 'y','pass_cross','pass_switch','technique_name','pass_goal_assist','body_part_name']]

In [183]:
df_passes_sb.columns

Index(['play_pattern_name', 'pass_angle', 'pass_length', 'end_x', 'end_y',
       'sub_type_name', 'x', 'y', 'pass_cross', 'pass_switch',
       'technique_name', 'pass_goal_assist', 'body_part_name'],
      dtype='object')

In [184]:
df_passes_sb.shape

(39214, 13)

In [185]:
df_passes_sb['pass_goal_assist'].unique()

array([nan, True], dtype=object)

In [186]:
df_passes_sb['pass_goal_assist'] = df_passes_sb['pass_goal_assist'].fillna(False)

In [187]:
# Rename columns to match previous dataset
df_passes_sb = df_passes_sb.rename(columns={
    'play_pattern_name': 'Play_Pattern',
    'pass_angle': 'pass_angle',
    'pass_length': 'pass_distance',
    'end_x': 'endX',
    'end_y': 'endY',
    'sub_type_name': 'Pass_Type',
    'x': 'x',
    'y': 'y',
    'pass_goal_assist': 'Is_Assist'
})

# Convert 'Is_Assist' to boolean
df_passes_sb['Is_Assist'] = df_passes_sb['Is_Assist'].astype(bool)


df_passes_sb['Pass_Type'] = 'ground_pass'
df_passes_sb.loc[df_passes_sb['pass_cross'] == 1, 'Pass_Type'] = 'cross'
df_passes_sb.loc[df_passes_sb['pass_switch'] == 1, 'Pass_Type'] = 'long_ball'
df_passes_sb.loc[df_passes_sb['technique_name'] == 'Through Ball', 'Pass_Type'] = 'through_pass'

# Map Play_Pattern categories from 'play_pattern_name'
play_pattern_mapping = {
    'Regular Play': 'open_play',
    'From Free Kick': 'freekick',
    'From Throw In': 'throw_in',
    'From Corner': 'corner'
}

# Apply the mapping
df_passes_sb['Play_Pattern'] = df_passes_sb['Play_Pattern'].map(play_pattern_mapping).fillna('other')

# Ensure column order matches old dataset
final_df_passes_sb = df_passes_sb[['x', 'y', 'endX', 'endY', 'pass_distance', 'pass_angle',
                   'Is_Assist', 'Pass_Type', 'Play_Pattern']]

In [188]:
final_df = final_df_passes_sb.copy()

In [189]:
final_df['Play_Pattern'].unique()

array(['other', 'open_play', 'freekick', 'throw_in', 'corner'],
      dtype=object)

## Feature Engg

In [207]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Ensure no missing values
final_df['Pass_Type'].fillna('Unknown', inplace=True)
final_df['Play_Pattern'].fillna('Unknown', inplace=True)

# One-hot encode categorical variables
categorical_features = ['Pass_Type', 'Play_Pattern']
encoder = OneHotEncoder(sparse_output=False, drop='first')

encoded_features = encoder.fit_transform(final_df[categorical_features])

# Convert to DataFrame & ensure index matches df_passes
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features), index=final_df.index)

# Normalize numerical features
scaler = MinMaxScaler()
final_df[['x', 'y', 'endX', 'endY', 'pass_distance', 'pass_angle']] = scaler.fit_transform(
    final_df[['x', 'y', 'endX', 'endY', 'pass_distance', 'pass_angle']]
)

# Combine all features (ensure same row count)
df_model = pd.concat([final_df[['x', 'y', 'endX', 'endY', 'pass_distance', 'pass_angle']], encoded_df], axis=1)


C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3464117837.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Pass_Type'].fillna('Unknown', inplace=True)
C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3464117837.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Play_Pattern'].fillna('Unknown', inplace=True)
C:\Users\acer\AppData\Local\Temp\ipykernel_23116\3464117837.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

## Training Model

In [208]:
final_df['Is_Assist'].unique()

array([False,  True])

In [211]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Define target variable (convert boolean to float)
X = df_model
y = final_df['Is_Assist'].astype(float)  # Treat as continuous

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict Expected Assists (xA) as a continuous value
final_df['expected_assist'] = model.predict(X)

# Evaluate model performance
y_pred = model.predict(X_test)
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_23116\4165391419.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['expected_assist'] = model.predict(X)


Mean Absolute Error: 0.004749701008030071


In [212]:
import joblib

# Save the trained model
joblib.dump(model, 'expected_assist_model.pkl')

['expected_assist_model.pkl']

In [213]:
final_df[final_df['Is_Assist'] == True].head()

,x,y,outcomeType,endX,endY,name,pass_angle,Is_Assist,Pass_Type,Play_Pattern,pass_distance,expected_assist
281,0.844601,0.601,Successful,0.872449,0.449449,Bernardo Silva,0.281821,True,ground_pass,open_play,0.117378,0.69
1687,0.643794,0.284,Successful,0.803061,0.439439,Ben Johnson,0.593685,True,through_pass,open_play,0.219983,0.66
1745,0.449041,0.917,Successful,0.806122,0.711712,Kevin De Bruyne,0.441619,True,through_pass,open_play,0.457928,0.69
3295,0.708375,0.352,Successful,0.813265,0.434434,Bernardo Silva,0.580368,True,ground_pass,open_play,0.131269,0.00
3605,0.820383,0.484,Successful,0.886735,0.397397,Rico Lewis,0.377977,True,ground_pass,open_play,0.093059,0.63
